### Preparación del Dataset

Las imágenes se extraen de el archivo ZIP especificado con las 40384 y se procesan mediante varias transformaciones:

Redimensionar: Cada imagen se escala a un tamaño uniforme de 224x224 píxeles.

Conversión a escala de grises: Las imágenes de 3 canales (RGB) se convierten a un solo canal (grises).

Aplicar CLAHE: Se mejora el contraste de las imágenes utilizando el método de ecualización adaptativa de histograma limitado por contraste(CLAHE), visto en Vision Por Computador.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import zipfile
import cv2
import numpy as np

# Ruta al archivo ZIP original
zip_file_path = '/content/drive/MyDrive/Imagenes IA/AugmentedAlzheimerDataset.zip'

# Ruta del nuevo archivo ZIP donde se guardarán las imágenes procesadas
output_zip_file_path = '/content/drive/MyDrive/Imagenes IA/SimplifiedProcessedAlzheimerDataset.zip'

# Subdirectorio donde están las clases
subdir = "AugmentedAlzheimerDataset"

# Función para aplicar CLAHE, redimensionar y normalizar las imágenes
def apply_clahe_resize_and_normalize(image):
    # Redimensionar la imagen a 224x224
    resized_image = cv2.resize(image, (224, 224))

    # Convertir la imagen a escala de grises
    gray_image = cv2.cvtColor(resized_image, cv2.COLOR_BGR2GRAY)

    # Aplicar CLAHE
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    clahe_image = clahe.apply(gray_image)

    # Normalizar la imagen (escala de 0 a 1)
    normalized_image = clahe_image / 255.0  # Normalizar a [0, 1]

    return normalized_image

# Procesar y guardar las imágenes en un nuevo archivo ZIP
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    with zipfile.ZipFile(output_zip_file_path, 'w') as output_zip_ref:
        for file_name in zip_ref.namelist():
            # Filtrar archivos dentro del subdirectorio especificado
            if file_name.startswith(f"{subdir}/") and file_name.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp')):
                # Extraer la clase del archivo (carpeta dentro del subdirectorio)
                relative_path = file_name[len(subdir) + 1:]  # Quitar el subdirectorio del path
                if '/' in relative_path:
                    class_name, file_basename = relative_path.split('/', 1)  # Obtener clase y nombre del archivo

                    # Leer la imagen
                    with zip_ref.open(file_name) as file:
                        img_data = file.read()
                        img_array = np.asarray(bytearray(img_data), dtype=np.uint8)
                        img = cv2.imdecode(img_array, cv2.IMREAD_COLOR)

                        if img is not None:
                            # Aplicar las transformaciones
                            processed_img = apply_clahe_resize_and_normalize(img)

                            # Convertir imagen procesada a bytes
                            _, img_encoded = cv2.imencode('.jpg', processed_img * 255)  # Escalar de [0, 1] a [0, 255]
                            img_bytes = img_encoded.tobytes()

                            # Guardar en el nuevo ZIP bajo el nivel directo de clases
                            new_file_name = f"{class_name}/{file_basename}"
                            output_zip_ref.writestr(new_file_name, img_bytes)

print(f"Imágenes procesadas guardadas en el nuevo archivo ZIP: {output_zip_file_path}")


Imágenes procesadas guardadas en el nuevo archivo ZIP: /content/drive/MyDrive/Imagenes IA/SimplifiedProcessedAlzheimerDataset.zip


Las imágenes transformadas se guardan en un nuevo archivo ZIP, organizado por clases según el directorio original.